In [1]:
import gdown
!pip install streamlit
!pip install pyngrok
gdown.download(id = '1-ZINL1PrDICvEJru1ZU27KiUdVeJnBVD')
!unzip ngrok-stable-linux-amd64.zip
import streamlit as st
import gdown
import pickle
import tensorflow as tf
import os
from keras.models import load_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
nltk.download('stopwords')
stop_words = stopwords.words('english')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.0 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=48c307b5b981de8a865632514846f771853ff4a71c601929058353b862882d23
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

Downloading...
From: https://drive.google.com/uc?id=1-ZINL1PrDICvEJru1ZU27KiUdVeJnBVD
To: /content/ngrok-stable-linux-amd64.zip
100%|██████████| 13.9M/13.9M [00:00<00:00, 33.6MB/s]


Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [7]:
%% writefile app.py

import streamlit as st
import gdown
import pickle
import tensorflow as tf
import os
from keras.models import load_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from wordcloud import WordCloud
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import re

os.makedirs('content', exist_ok=True)
os.chdir('content')

@st.cache_data
def collect():
  nltk.download('stopwords')
  stop_words = stopwords.words('english')
  nltk.download('omw-1.4')
  nltk.download('punkt')
  nltk.download('wordnet')
  nltk.download('averaged_perceptron_tagger')
  gdown.download(id = '1zajG3hnHRbzRPO7Sv_pJC5fsfiipMUk0')
  return

@st.cache_resource
def getdata():

  gdown.download(id = '1qFViB4VmxIrC_atqunNzq9pqUaz0YMk0')
  gdown.download(id = '1YGzqZhxig_5szsufoDXYJdRiAvxKtz1F')
  
  with open('/content/content/tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)
  model = load_model('/content/content/model.h5')
  
  return tokenizer, model

def set_background(theme):

    encoded_image = 0
    
    page_bg_css = f'''
        <style>
            body {{
                background-image: url("data:image/jpeg;base64,{encoded_image}");
                background-size: cover;
            }}

            .stApp {{
                color: {theme['textColor']};
                background-color: {theme['backgroundColor']};
            }}
        </style>
    '''
    st.markdown(page_bg_css, unsafe_allow_html=True)

tokenizer, model = getdata()

lemmatizer = WordNetLemmatizer()

def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))  

    lemmatized_sentence = []
    
    for word, tag in nltk_tagged:
        if tag is None:

            lemmatized_sentence.append(word)
        else:
          try:
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
          except:
            lemmatized_sentence.append(word)

    return " ".join(lemmatized_sentence) 


def badword_prediction(input_data):
     
    df = pd.DataFrame(columns = ['text'])
    new_row= {'text': input_data}
    new_row = pd.DataFrame(new_row, index=[0])
    df = pd.concat([df, new_row], ignore_index=True)
    
    df['text'] = [str(i).lower() for i in df['text']]
    df['text'] = df['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', x))
    
    df['text'] = df['text'].apply(lambda x: lemmatize_sentence(x))
    
    texts = df['text'].to_list()

    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    
    max_sequence_length = 15 # based on pad_seq length sugestion
    padded_sequence = pad_sequences(sequences, maxlen=max_sequence_length)

    predictions = model.predict(padded_sequence)[0][0]
    threshold = 0.7
    predictions = np.exp(predictions) / (np.exp(predictions) + 1)
    confidence = predictions
    predictions = ('it is a Bad word with a confidence of '+str(predictions)+' %' if predictions >= threshold else 'it is not a Bad word' )
       
    return(predictions)

def main():
    a = collect()
    theme = {
        'base': 'dark',
        'backgroundColor': '#005C1D',
        'textColor': '#ffffff'
    }

    set_background(theme)

    st.write('Welcome to the, *Profanity Checker Application* :sunglasses:')
    st.title('Check your sentence')
    st.text('To get better results try typing words with atleast a length of 7')
    # input
    text = st.text_input('Enter your text:')
    
    # prediction
    ans = ""
    
    # button
    if st.button('Check'):
        ans = badword_prediction(text)
        
    st.success(ans)
    
    link = st.expander('*Project Link!* :point_down:', expanded=False)
    
    with link:
        st.markdown('Visit my project in this [link](https://github.com/Selvamsmail/Profinity_Classification)')


if __name__ == '__main__':
    main()

Overwriting app.py


In [9]:
!ngrok authtoken 2QXqS75INPMpgJ6YdQw8naw6LlS_3zUyKYqm9SbVUnRckc5QP
get_ipython().system_raw('./ngrok http 8501 &')
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
https://f4eb-35-196-254-200.ngrok-free.app


In [10]:
!streamlit run /content/app.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.254.200:8501

2023-05-31 16:01:52.197000: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Downloading...
From: https://drive.google.com/uc?id=1qFViB4VmxIrC_atqunNzq9pqUaz0YMk0
To: /content/content/model.h5
100% 114M/114M [00:00<00:00, 139MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YGzqZhxig_5szsufoDXYJdRiAvxKtz1F
To: /content/content/tokenizer.pickle
100% 4.02M/4.02M [00:00<00:00, 143MB/s]
1/1 [==============================] - 0s 162ms/step
  Stopping...
  Stopping...


In [27]:

# # Load the tokenizer
# with open('C:/Users/PM/.spyder-py3/content/tokenizer.pickle', 'rb') as handle:
#     tokenizer = pickle.load(handle)

# # Load the trained model
# model = load_model('C:/Users/PM/.spyder-py3/content/model.h5')

In [28]:
# def remove_stopwords(rev):

#     review_tokenized = word_tokenize(rev)
#     rev_new = " ".join([i for i in review_tokenized  if i not in stop_words])
#     return rev_new   

In [11]:
!pip install pipreqs
!pipreqs ./ --force

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
